In [1]:
import os
import logging
import pickle
import json
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from gensim.models import Phrases, LdaModel
from gensim.corpora import Dictionary
from gensim.models import CoherenceModel
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
from pprint import pprint
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import pandas as pd

class TopicModeler:
    def __init__(self, input_path):
        self.input_path = input_path
        self.docs = []
        self.filtered_docs = []
        self.dictionary = None
        self.corpus = None
        self.model = None
        self.coherence_scores = []
        self.tokenizer = RegexpTokenizer(r'\w+')
        self.lemmatizer = WordNetLemmatizer()
        self.high_freq_words = set()
        logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

    def load_documents(self):
        for filename in os.listdir(self.input_path):
            file_path = os.path.join(self.input_path, filename)
            if os.path.isfile(file_path) and filename.endswith('.txt'):
                with open(file_path, 'r', encoding='utf-8') as f:
                    content = f.read().strip()
                    if isinstance(content, str):
                        self.docs.append(content)
        print(f"Loaded {len(self.docs)} documents")
        if self.docs:
            print(f"Sample document: {self.docs[0][:100]}...")

    def load_and_filter_high_frequency_words(self, csv_folder='.'):
        all_terms = pd.DataFrame()

        for i in range(1, 6):
            file_path = os.path.join(csv_folder, f'sorted_terms_topic_{i}.csv')
            if os.path.exists(file_path):
                df = pd.read_csv(file_path)
                all_terms = pd.concat([all_terms, df])

        grouped_terms = all_terms.groupby('Term')['Total'].sum().reset_index()
        self.high_freq_words = set(grouped_terms[grouped_terms['Total'] > 10]['Term'])
        print(f"Identified {len(self.high_freq_words)} high-frequency words to remove")

    def preprocess_documents(self):
        self.load_and_filter_high_frequency_words()

        self.docs = [[token.lower() for token in self.tokenizer.tokenize(doc)] for doc in self.docs if isinstance(doc, str)]
        self.docs = [[token for token in doc if not token.isnumeric() and len(token) > 1] for doc in self.docs]
        stop_words = set(stopwords.words('english'))
        self.docs = [[word for word in doc if word not in stop_words and word not in self.high_freq_words] for doc in self.docs]
        self.docs = [[self.lemmatizer.lemmatize(token) for token in doc] for doc in self.docs]
        print(f"After preprocessing: {len(self.docs)} documents")
        if self.docs:
            print(f"Sample preprocessed document: {self.docs[0][:10]}...")

    def add_bigrams(self):
        bigram = Phrases(self.docs, min_count=5, threshold=100)
        for idx in range(len(self.docs)):
            for token in bigram[self.docs[idx]]:
                if '_' in token:
                    self.docs[idx].append(token)
        print(f"Added bigrams. Sample document: {self.docs[0][:15]}...")

    def filter_with_tfidf(self, top_n=200, bottom_n=50):
        texts = [' '.join(doc) for doc in self.docs]
        tfidf_vectorizer = TfidfVectorizer()
        tfidf_matrix = tfidf_vectorizer.fit_transform(texts)
        feature_names = tfidf_vectorizer.get_feature_names_out()
        mean_tfidf_scores = np.asarray(tfidf_matrix.mean(axis=0)).flatten()
        
        top_indices = mean_tfidf_scores.argsort()[-top_n:]
        bottom_indices = mean_tfidf_scores.argsort()[:bottom_n]
        
        words_to_remove = set(feature_names[i] for i in np.concatenate([top_indices, bottom_indices]))

        self.filtered_docs = [[word for word in doc if word not in words_to_remove] for doc in self.docs]
        print(f"Filtered documents using TF-IDF. Sample filtered document: {self.filtered_docs[0][:10]}...")

    def create_dictionary_and_corpus(self):
        self.dictionary = Dictionary(self.filtered_docs)
        original_size = len(self.dictionary)
        self.dictionary.filter_extremes(no_below=2, no_above=0.9)
        self.corpus = [self.dictionary.doc2bow(doc) for doc in self.filtered_docs]
        print(f"Dictionary size: {len(self.dictionary)} (reduced from {original_size})")
        print(f"Corpus size: {len(self.corpus)}")
        if self.corpus:
            print(f"Sample corpus entry: {self.corpus[0][:10]}")

    def train_lda_model(self, num_topics=6, chunksize=2000, passes=20, iterations=400, alpha='auto', eta='auto'):
        if not self.corpus or not self.dictionary:
            raise ValueError("Corpus or dictionary is empty. Check your preprocessing steps.")
        
        print(f"Training LDA model with num_topics={num_topics}, chunksize={chunksize}, passes={passes}, iterations={iterations}, alpha={alpha}, eta={eta}")
        
        self.model = LdaModel(
            corpus=self.corpus,
            id2word=self.dictionary,
            chunksize=chunksize,
            alpha=alpha,
            eta=eta,
            iterations=iterations,
            num_topics=num_topics,
            passes=passes,
            eval_every=None,
            random_state=42
        )
        print("LDA model trained successfully")

    def print_model_info(self):
        print('Number of unique tokens: %d' % len(self.dictionary))
        print('Number of documents: %d' % len(self.corpus))
        top_topics = self.model.top_topics(self.corpus)
        avg_topic_coherence = sum([t[1] for t in top_topics]) / self.model.num_topics
        print('Average topic coherence: %.4f.' % avg_topic_coherence)
        print("\nTop topics:")
        pprint(self.model.print_topics())
        print('\nPerplexity: ', self.model.log_perplexity(self.corpus))
        coherence_model_lda = CoherenceModel(model=self.model, texts=self.filtered_docs, dictionary=self.dictionary, coherence='c_v')
        coherence_lda = coherence_model_lda.get_coherence()
        print('\nCoherence Score: ', coherence_lda)
        return coherence_lda

    def visualize_topics(self):
        vis = pyLDAvis.gensim.prepare(self.model, self.corpus, self.dictionary)
        for i in range(1, 6):
            df = vis.sorted_terms(topic=i, _lambda=0)
            df.to_csv(f'sorted_terms_topic_{i}.csv', index=False)
        return vis

    def analyze_documents(self):
        for i, doc in enumerate(self.filtered_docs[:5]):
            bow = self.dictionary.doc2bow(doc)
            doc_topics = self.model.get_document_topics(bow)
            print(f"\nDocument {i} topics:")
            pprint(doc_topics)
            best_topic = max(doc_topics, key=lambda x: x[1])
            print(f"Best topic: {best_topic[0]}")
            print(f"Top words in this topic:")
            pprint(self.model.show_topic(best_topic[0]))
            print(f"Original document: {' '.join(doc[:30])}...")
            print()

    def save_model(self, file_path):
        if self.model is None:
            raise ValueError("No model has been trained yet.")
        os.makedirs(os.path.dirname(file_path), exist_ok=True)
        self.model.save(file_path)
        print(f"Model saved to {file_path}")

    def load_model(self, file_path):
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"No model file found at {file_path}")
        self.model = LdaModel.load(file_path)
        print(f"Model loaded from {file_path}")

    def save_dictionary_and_corpus(self, dict_path, corpus_path):
        if self.dictionary is None or self.corpus is None:
            raise ValueError("Dictionary and corpus have not been created yet.")
        os.makedirs(os.path.dirname(dict_path), exist_ok=True)
        os.makedirs(os.path.dirname(corpus_path), exist_ok=True)
        self.dictionary.save(dict_path)
        with open(corpus_path, 'wb') as f:
            pickle.dump(self.corpus, f)
        print(f"Dictionary saved to {dict_path}")
        print(f"Corpus saved to {corpus_path}")

    def load_dictionary_and_corpus(self, dict_path, corpus_path):
        if not os.path.exists(dict_path) or not os.path.exists(corpus_path):
            raise FileNotFoundError(f"Dictionary or corpus file not found.")
        self.dictionary = Dictionary.load(dict_path)
        with open(corpus_path, 'rb') as f:
            self.corpus = pickle.load(f)
        print(f"Dictionary loaded from {dict_path}")
        print(f"Corpus loaded from {corpus_path}")

    def plot_coherence_scores(self, iteration):
        num_topics = [x[0] for x in self.coherence_scores]
        coherences = [x[1] for x in self.coherence_scores]
        
        plt.figure(figsize=(10, 6))
        plt.plot(num_topics, coherences, marker='o')
        plt.xlabel('Number of Topics')
        plt.ylabel('Coherence Score')
        plt.title('Coherence Score by Number of Topics')
        plt.grid(True)
        
        plt.xticks(num_topics)
        plt.ylim(0, max(coherences) * 1.1)
        
        plt.savefig(f'coherence_scores_iteration_{iteration}.png')
        plt.close()

    def run_with_params(self, num_topics, chunksize, passes, iterations):
        try:
            self.load_documents()
            self.preprocess_documents()
            self.add_bigrams()
            self.filter_with_tfidf(top_n=200, bottom_n=50)
            self.create_dictionary_and_corpus()
            self.train_lda_model(num_topics=num_topics, chunksize=chunksize, passes=passes, iterations=iterations)
            model_dir = f"lda_models/topics_{num_topics}_passes_{passes}_iterations_{iterations}"
            self.save_model(os.path.join(model_dir, 'trained_model'))
            self.save_dictionary_and_corpus(os.path.join(model_dir, 'dictionary'), os.path.join(model_dir, 'corpus'))
            coherence_lda = self.print_model_info()
            self.coherence_scores.append((num_topics, coherence_lda))
            
            self.plot_coherence_scores(len(self.coherence_scores))
            
            vis = self.visualize_topics()
            pyLDAvis.save_html(vis, os.path.join(model_dir, 'lda_visualization.html'))
            self.analyze_documents()
            
            return vis
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            raise

if __name__ == "__main__":
    modeler = TopicModeler('txt')

    param_combinations = [
        (4, 2000, 10, 200),
        (8, 2000, 10, 400),
        (12, 2000, 20, 200),
        (16, 2000, 20, 400),
        (20, 2000, 20, 400),
        (24, 2000, 20, 400)
    ]

    for num_topics, chunksize, passes, iterations in param_combinations:
        print(f"\nRunning model with parameters: num_topics={num_topics}, chunksize={chunksize}, passes={passes}, iterations={iterations}")
        modeler.run_with_params(num_topics, chunksize, passes, iterations)


Running model with parameters: num_topics=4, chunksize=2000, passes=10, iterations=200
Loaded 440 documents
Sample document: Pricing for interconnection related to the provision of number portability, as referred to in Articl...
Identified 132 high-frequency words to remove


2024-08-10 12:14:40,024 : INFO : collecting all words and their counts
2024-08-10 12:14:40,024 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2024-08-10 12:14:40,043 : INFO : collected 21968 token types (unigram + bigrams) from a corpus of 41938 words and 440 sentences
2024-08-10 12:14:40,043 : INFO : merged Phrases<21968 vocab, min_count=5, threshold=100, max_vocab_size=40000000>
2024-08-10 12:14:40,044 : INFO : Phrases lifecycle event {'msg': 'built Phrases<21968 vocab, min_count=5, threshold=100, max_vocab_size=40000000> in 0.02s', 'datetime': '2024-08-10T12:14:40.044280', 'gensim': '4.3.2', 'python': '3.11.9 (main, Jul 16 2024, 11:07:54) [Clang 14.0.0 (clang-1400.0.29.202)]', 'platform': 'macOS-12.5-arm64-arm-64bit', 'event': 'created'}
2024-08-10 12:14:40,180 : INFO : adding document #0 to Dictionary<0 unique tokens: []>
2024-08-10 12:14:40,187 : INFO : built Dictionary<3302 unique tokens: ['abstract', 'adoption', 'advance', 'appeal', 'calculating']...> from

After preprocessing: 440 documents
Sample preprocessed document: ['pricing', 'interconnection', 'related', 'provision', 'portability', 'referred', 'article', 'directive', 'ec', 'european']...
Added bigrams. Sample document: ['pricing', 'interconnection', 'related', 'provision', 'portability', 'referred', 'article', 'directive', 'ec', 'european', 'parliament', 'council', 'march', 'universal', 'service']...
Filtered documents using TF-IDF. Sample filtered document: ['pricing', 'interconnection', 'related', 'portability', 'concern', 'traffic', 'number', 'ported', 'incurred', 'mobile']...
Dictionary size: 1715 (reduced from 3302)
Corpus size: 440
Sample corpus entry: [(0, 1), (1, 1), (2, 3), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1)]
Training LDA model with num_topics=4, chunksize=2000, passes=10, iterations=200, alpha=auto, eta=auto


2024-08-10 12:14:40,353 : INFO : optimized alpha [0.12164946, 0.10884525, 0.11331521, 0.121583626]
2024-08-10 12:14:40,354 : INFO : topic #0 (0.122): 0.005*"property" + 0.005*"le" + 0.005*"currency" + 0.005*"provider" + 0.004*"data" + 0.004*"also" + 0.004*"et" + 0.004*"foreign" + 0.004*"established" + 0.004*"purchase"
2024-08-10 12:14:40,354 : INFO : topic #1 (0.109): 0.006*"mortgage" + 0.005*"make" + 0.005*"provider" + 0.005*"whose" + 0.004*"compensation" + 0.004*"determine" + 0.004*"intended" + 0.004*"creditor" + 0.004*"property" + 0.004*"consequence"
2024-08-10 12:14:40,355 : INFO : topic #2 (0.113): 0.008*"trade_mark" + 0.005*"significant" + 0.004*"ascertain" + 0.004*"ass" + 0.004*"travel" + 0.004*"mean" + 0.004*"also" + 0.004*"currency" + 0.003*"concept" + 0.003*"unfairness"
2024-08-10 12:14:40,355 : INFO : topic #3 (0.122): 0.005*"damage" + 0.004*"call" + 0.004*"device" + 0.004*"italian" + 0.004*"producer" + 0.004*"form" + 0.004*"vehicle" + 0.004*"trade_mark" + 0.004*"however" + 

LDA model trained successfully
Model saved to lda_models/topics_4_passes_10_iterations_200/trained_model
Dictionary saved to lda_models/topics_4_passes_10_iterations_200/dictionary
Corpus saved to lda_models/topics_4_passes_10_iterations_200/corpus
Number of unique tokens: 1715
Number of documents: 440
Average topic coherence: -11.6718.

Top topics:
[(0,
  '0.008*"property" + 0.007*"le" + 0.007*"provider" + 0.007*"et" + '
  '0.005*"purchase" + 0.005*"data" + 0.005*"assurance" + 0.005*"established" + '
  '0.005*"la" + 0.005*"premise"'),
 (1,
  '0.006*"mortgage" + 0.005*"compensation" + 0.005*"creditor" + 0.005*"make" + '
  '0.005*"consequence" + 0.005*"lawyer" + 0.004*"whose" + 0.004*"provider" + '
  '0.004*"organic" + 0.004*"based"'),
 (2,
  '0.009*"trade_mark" + 0.008*"currency" + 0.006*"foreign" + 0.006*"travel" + '
  '0.005*"significant" + 0.005*"unfairness" + 0.005*"ass" + 0.005*"informed" + '
  '0.005*"financial" + 0.005*"exchange"'),
 (3,
  '0.007*"damage" + 0.006*"trade_mark" + 

...


Document 0 topics:
[(18, 0.9881489)]
Best topic: 18
Top words in this topic:
[('directory', 0.025856007),
 ('telephone', 0.017536804),
 ('subscriber', 0.01618872),
 ('request', 0.016156826),
 ('making', 0.01585196),
 ('data', 0.014585703),
 ('energy', 0.012974871),
 ('instrument', 0.012870217),
 ('label', 0.009751267),
 ('enquiry', 0.0097427685)]
Original document: pricing interconnection related portability concern traffic number ported incurred telephone operator implement request porting adoption method calculating fix advance abstract model maximum charged donor recipient fixed way dissuaded making facility...


Document 1 topics:
[(5, 0.9790661)]
Best topic: 5
Top words in this topic:
[('building', 0.016450152),
 ('passenger', 0.01579408),
 ('natural', 0.014967524),
 ('long', 0.013274875),
 ('flight', 0.011989219),
 ('common', 0.011787071),
 ('energy', 0.010504),
 ('procedure', 0.010503147),
 ('cancellation', 0.010498826),
 ('need', 0.01049444)]
Original document: natural becomes

2024-08-10 12:14:57,145 : INFO : using autotuned alpha, starting with [0.041666668, 0.041666668, 0.041666668, 0.041666668, 0.041666668, 0.041666668, 0.041666668, 0.041666668, 0.041666668, 0.041666668, 0.041666668, 0.041666668, 0.041666668, 0.041666668, 0.041666668, 0.041666668, 0.041666668, 0.041666668, 0.041666668, 0.041666668, 0.041666668, 0.041666668, 0.041666668, 0.041666668]
2024-08-10 12:14:57,146 : INFO : using serial LDA version on this node
2024-08-10 12:14:57,148 : INFO : running online (multi-pass) LDA training, 24 topics, 20 passes over the supplied corpus of 440 documents, updating model once every 440 documents, evaluating perplexity every 0 documents, iterating 400x with a convergence threshold of 0.001000
2024-08-10 12:14:57,148 : INFO : PROGRESS: pass 0, at document #440/440
2024-08-10 12:14:57,294 : INFO : optimized alpha [0.037308037, 0.03676772, 0.036728077, 0.03640121, 0.037278034, 0.036961325, 0.036847305, 0.036535025, 0.036751486, 0.037272654, 0.03770063, 0.03694

LDA model trained successfully
Model saved to lda_models/topics_24_passes_20_iterations_400/trained_model
Dictionary saved to lda_models/topics_24_passes_20_iterations_400/dictionary
Corpus saved to lda_models/topics_24_passes_20_iterations_400/corpus
Number of unique tokens: 1724
Number of documents: 440
Average topic coherence: -10.6526.

Top topics:
[(23,
  '0.011*"note" + 0.011*"access" + 0.011*"compensation" + 0.010*"required" + '
  '0.008*"set" + 0.008*"arising" + 0.008*"technical" + 0.008*"complaint" + '
  '0.008*"slovak" + 0.008*"communication_network"'),
 (7,
  '0.017*"reasonably" + 0.012*"provide" + 0.010*"travel" + 0.010*"operation" + '
  '0.008*"reimbursement" + 0.008*"brought" + 0.007*"context" + '
  '0.007*"unfairness" + 0.007*"cheese" + 0.007*"force"'),
 (13,
  '0.017*"compensation" + 0.014*"appeal" + 0.012*"defect" + 0.012*"additional" '
  '+ 0.012*"mandatory" + 0.010*"disease" + 0.010*"link" + 0.008*"existence" + '
  '0.008*"considered" + 0.008*"operator"'),
 (2,
  '0.

2024-08-10 12:14:59,894 : INFO : 9 accumulators retrieved from output queue
2024-08-10 12:14:59,962 : INFO : accumulated word occurrence stats for 1368 virtual documents



Coherence Score:  0.42170033573309224

Document 0 topics:
[(6, 0.9910919)]
Best topic: 6
Top words in this topic:
[('email', 0.016154146),
 ('number', 0.010794199),
 ('january', 0.010794195),
 ('regulatory', 0.009007549),
 ('operator', 0.009007549),
 ('insurance', 0.009000349),
 ('covered', 0.008835581),
 ('appeal', 0.007583737),
 ('framework', 0.0072208983),
 ('display', 0.0072208983)]
Original document: pricing interconnection related number portability concern traffic number ported set incurred mobile telephone operator implement request number porting adoption laying method used calculating fix advance abstract model maximum charged donor...


Document 1 topics:
[(22, 0.9725119)]
Best topic: 22
Top words in this topic:
[('vehicle', 0.021714468),
 ('device', 0.020505369),
 ('cancellation', 0.020483933),
 ('energy', 0.016916452),
 ('passenger', 0.01690911),
 ('protect', 0.0131267635),
 ('outside', 0.011961081),
 ('investment', 0.01175638),
 ('event', 0.011729252),
 ('flight', 0.0096

In [ ]:
docs/
├── .vitepress/
│   └── config.ts
├── index.md
├── guide/
│   ├── installation.md
│   ├── getting-started.md
│   └── configuration.md
├── methodology/
│   ├── preprocessing.md
│   ├── filtering.md
│   ├── model-training.md
│   └── visualization.md
└── api/
    └── topic-modeler.md

In [ ]:
docs/
├── .vitepress/
│   └── config.ts
├── index.md
├── guide/
│   ├── installation.md
│   ├── getting-started.md
│   └── configuration.md
├── methodology/
│   ├── preprocessing.md
│   ├── filtering.md
│   ├── model-training.md
│   └── visualization.md
└── api/
    └── topic-modeler.md